In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import cv2
import numpy as np

def load_depth_csv(path: Path, depth_scale: float = 0.001) -> np.ndarray:
    """
    CSV形式の深度画像を読み込み，メートル[m]単位に変換して返す関数

    Parameters
    ----------
    path : Path
        深度CSVファイルへのパス
    depth_scale : float
        元の値に掛けるスケール係数
        例）RealSenseのuint16[mm]なら 0.001 を掛けて[m]にする

    Returns
    -------
    depth_m : np.ndarray
        深度[m]（0以下はNaNで無効扱い）
    """
    # CSVを読み込み（float32に変換）
    d = np.loadtxt(path, delimiter=",").astype(np.float32)
    # 0以下の値は無効値としてNaNにしてお
    d[d <= 0] = np.nan
    # スケールを掛けて[m]に変換
    return d * depth_scale

def depth_to_colormap(depth_m: np.ndarray, near: float | None = None, far: float | None = None):
    """
    深度[m]画像をヒートマップ（BGRカラー）に変換する関数

    Parameters
    ----------
    depth_m : np.ndarray
        深度[m]（NaNは無効値）
    near : float | None
        カラーマップに使う手前側の距離[m]
        Noneなら有効画素の5パーセンタイルから自動推定
    far : float | None
        カラーマップに使う奥側の距離[m]
        Noneなら有効画素の95パーセンタイルから自動推定

    Returns
    -------
    depth_cmap : np.ndarray
        COLORMAP_JETでカラー化したBGR画像
    near : float
        実際に使用したnear値
    far : float
        実際に使用したfar値
    """

    # 有効な深度値（NaN以外）だけ取得
    valid = depth_m[np.isfinite(depth_m)]

    # near / far が指定されていない場合は自動推定
    if near is None or far is None:
        if valid.size == 0:
            # すべて無効値だった場合のデフォルトレンジ
            near, far = 0.2, 1.0
        else:
            # 5〜95パーセンタイルをレンジにする
            near = float(np.percentile(valid, 5))
            far  = float(np.percentile(valid, 95))
            # near = far になってしまう場合の対策
            if near == far:
                near, far = max(near - 0.05, 0.05), near + 0.05
    
    # 深度を [0,1] に正規化した後，[0,255] のuint8にスケーリング
    x = np.clip((depth_m - near) / (far - near), 0, 1)
    vis = (x * 255).astype(np.uint8)

    # OpenCVのカラーマップでヒートマップ化（BGR）
    depth_cmap = cv2.applyColorMap(vis, cv2.COLORMAP_JET)
    return depth_cmap, near, far

def show_overlay_inline(
    rgb_path: Path,
    depth_csv_path: Path,
    alpha: float = 0.4,
    near: float | None = None,
    far: float | None = None,
    depth_scale: float = 0.001,
    save_dir: Path | None = None,
):
    """
    RGB画像と深度CSVを読み込み，ヒートマップとオーバーレイを
    Jupyter / Pythonスクリプト上で並べて表示する関数

    Parameters
    ----------
    rgb_path : Path
        RGB画像（BGRとして読み込む）のパス
    depth_csv_path : Path
        深度CSVファイルのパス
    alpha : float
        オーバーレイ時の深度ヒートマップの透過率（0〜1）
    near, far : float | None
        深度レンジ[m]。Noneなら自動推定
    depth_scale : float
        CSVの値 → [m] へのスケール係数
    save_dir : Path | None
        画像保存先ディレクトリ（Noneなら保存しない）

    Returns
    -------
    overlay : np.ndarray
        BGRのオーバーレイ画像
    depth_cmap : np.ndarray
        深度ヒートマップ（BGR）
    near_used : float
        使用したnear値
    far_used : float
        使用したfar値
    saved : dict
        保存したパス（"overlay", "panel", "depth_colormap"）を格納
    """

    # --- RGB画像読み込み（BGRとしてロード） ---
    color = cv2.imread(str(rgb_path))
    if color is None:
        raise FileNotFoundError(f"Could not read RGB image: {rgb_path}")

    # --- 深度CSV読み込み＆[m]変換 ---
    depth_m = load_depth_csv(depth_csv_path, depth_scale=depth_scale)

    # --- 解像度チェック（RGBと深度のサイズが一致しているか） ---
    if (color.shape[0], color.shape[1]) != depth_m.shape:
        raise ValueError(f"Size mismatch: RGB {color.shape[:2]} vs Depth {depth_m.shape}")

    # --- 深度をヒートマップ化 ---
    depth_cmap, near_used, far_used = depth_to_colormap(depth_m, near, far)

    # --- RGBと深度ヒートマップをαブレンド ---
    overlay = cv2.addWeighted(color, 1 - alpha, depth_cmap, alpha, 0)

    # ===== 可視化（matplotlibで並べて表示） =====
    # OpenCV(BGR) → matplotlib(RGB) へ変換
    color_rgb = cv2.cvtColor(color, cv2.COLOR_BGR2RGB)
    depth_rgb = cv2.cvtColor(depth_cmap, cv2.COLOR_BGR2RGB)
    overlay_rgb = cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(15,5))
    # 左：RGB
    plt.subplot(1,3,1); plt.imshow(color_rgb); plt.title("RGB"); plt.axis("off")
    # 中央：深度ヒートマップ（near / far をタイトルに表示）
    plt.subplot(1,3,2); plt.imshow(depth_rgb); plt.title(f"Depth (near={near_used:.2f}m, far={far_used:.2f}m)"); plt.axis("off")
     # 右：オーバーレイ
    plt.subplot(1,3,3); plt.imshow(overlay_rgb); plt.title("Overlay"); plt.axis("off")
    plt.show()

    # ===== 保存処理 =====
    saved = {}
    if save_dir is not None:
        save_dir = Path(save_dir)
        (save_dir / "overlay").mkdir(parents=True, exist_ok=True)
        (save_dir / "panel").mkdir(parents=True, exist_ok=True)
        (save_dir / "depth_colormap").mkdir(parents=True, exist_ok=True)  
        stem = rgb_path.stem
        overlay_path = save_dir / "overlay" / f"{stem}.png"
        panel_path = save_dir / "panel" / f"{stem}_panel.png"
        depth_path = save_dir / "depth_colormap" / f"{stem}_depth.png"    
        
        cv2.imwrite(str(overlay_path), overlay)
        cv2.imwrite(str(depth_path), depth_cmap)                         
        panel_img = np.hstack([color, depth_cmap, overlay])
        cv2.imwrite(str(panel_path), panel_img)
        saved["overlay"] = str(overlay_path)
        saved["panel"] = str(panel_path)
        saved["depth_colormap"] = str(depth_path)                        

    return overlay, depth_cmap, near_used, far_used, saved



In [2]:
print(Path("../datasets/train_data/color/2025-01-20T16-56-09.png").exists())

False


In [3]:
overlay, depth_cmap, near_used, far_used, saved = show_overlay_inline(
    rgb_path=Path("../datasets/train_data/color/2025-01-20T16-56-09.png"),
    depth_csv_path=Path("../datasets/train_data/csv_depth/2025-01-20T16-56-09.csv"),
    alpha=0.4,          # 重ね具合
    near=None, far=None,# 自動推定。固定したければmで指定
    depth_scale=0.001,  # RealSense: uint16 -> m
    #save_dir=None
    save_dir=Path("../datasets/train_data/")  # 保存したくなければ None
)

print("near/far used:", near_used, far_used)
print(saved)

[ WARN:0@13.460] global loadsave.cpp:244 findDecoder imread_('../datasets/train_data/color/2025-01-20T16-56-09.png'): can't open/read file: check file path/integrity


FileNotFoundError: Could not read RGB image: ../datasets/train_data/color/2025-01-20T16-56-09.png

/workspace/work/MDN_RND_Foods_Kaisei0504/datasets/train_data


In [ ]:
# make_overlay.py
import cv2, numpy as np
from pathlib import Path
import argparse

def load_depth_csv(path, depth_scale=0.001):
    d = np.loadtxt(path, delimiter=",").astype(np.float32)  # uint16を想定
    d[d <= 0] = np.nan
    return d * depth_scale  # meters

def depth_to_colormap(depth_m, near=None, far=None):
    valid = depth_m[np.isfinite(depth_m)]
    if near is None or far is None:
        if valid.size == 0:
            near, far = 0.2, 1.0
        else:
            near = np.percentile(valid, 5)
            far  = np.percentile(valid, 95)
            if near == far:  # 安全策
                near, far = max(near-0.05, 0.05), near+0.05
    x = np.clip((depth_m - near) / (far - near), 0, 1)
    vis = (x * 255).astype(np.uint8)
    return cv2.applyColorMap(vis, cv2.COLORMAP_JET), near, far

def main(rgb_path, depth_csv, out_path, alpha=0.4, near=None, far=None):
    color = cv2.imread(str(rgb_path))  # BGR
    depth_m = load_depth_csv(depth_csv)  # meters
    if (color.shape[0], color.shape[1]) != depth_m.shape:
        raise ValueError(f"Size mismatch: RGB {color.shape[:2]} vs Depth {depth_m.shape}")

    depth_cmap, near, far = depth_to_colormap(depth_m, near, far)
    overlay = cv2.addWeighted(color, 1 - alpha, depth_cmap, alpha, 0)

    # 参考に3連結も保存
    panel = np.hstack([color, depth_cmap, overlay])
    cv2.imwrite(str(out_path), overlay)
    cv2.imwrite(str(Path(out_path).with_name(Path(out_path).stem + "_panel.png")), panel)
    print(f"Saved: {out_path} (near={near:.3f}m, far={far:.3f}m)")

if __name__ == "__main__":
    ap = argparse.ArgumentParser()
    ap.add_argument("--rgb", required=True)
    ap.add_argument("--depth_csv", required=True)
    ap.add_argument("--out", default="overlay.png")
    ap.add_argument("--alpha", type=float, default=0.4)
    ap.add_argument("--near", type=float, default=None)
    ap.add_argument("--far",  type=float, default=None)
    args = ap.parse_args()
    main(args.rgb, args.depth_csv, args.out, args.alpha, args.near, args.far)